#### Solution
# Assignment 7 - Network Flow
## Part 2 - Edmonds-Karp

In [1]:
capacities = [
    #a, b, c, d, e, f
    [0, 0, 9, 9, 0, 0],  # a
    [9, 0, 0, 4, 5, 0],  # b
    [0, 9, 0, 0, 0, 0],  # c
    [0, 0, 0, 0, 9, 4],  # d
    [0, 0, 3, 0, 0, 9],  # e
    [0, 0, 5, 0, 0, 0],  # f
]
node_labels = ["a", "b", "c", "d", "e", "f"]

### Helper Functions

In [2]:
def to_str(node_index: int) -> str:
    return node_labels[node_index]


def to_index(node_label: str) -> int:
    return node_labels.index(node_label)


def path_to_str(path: list) -> str:
    """convert a path to a easy to read string of the visited nodes
    for instance: "abcdef"""
    if path is None:
        return "None"
    return "".join([to_str(node_index) for node_index in path])


# Testing the function
print(f"[0, 1, 2, 3, 4, 5] => {path_to_str([0, 1, 2, 3, 4, 5])}")


[0, 1, 2, 3, 4, 5] => abcdef


In [3]:
def duplicate(lst: list) -> list:
    """Duplicate/make copy of the list given,
    should be used to not edit original`capacities` during the algorithm
    se we can rerun each cell individually without resetting entire notebook"""
    return [row[:] for row in lst]


# Testing the function
print(f"{capacities}\n{duplicate(capacities)}")


[[0, 0, 9, 9, 0, 0], [9, 0, 0, 4, 5, 0], [0, 9, 0, 0, 0, 0], [0, 0, 0, 0, 9, 4], [0, 0, 3, 0, 0, 9], [0, 0, 5, 0, 0, 0]]
[[0, 0, 9, 9, 0, 0], [9, 0, 0, 4, 5, 0], [0, 9, 0, 0, 0, 0], [0, 0, 0, 0, 9, 4], [0, 0, 3, 0, 0, 9], [0, 0, 5, 0, 0, 0]]


### Implementing Edmonds-Karp

In [4]:
def BFS(C: list, start: int, end: int):
    shortest_paths = {start: [start]}
    visited, queue = [], [start]

    while queue != []:
        current_node = queue.pop(0)
        visited.append(current_node)

        # go through all possible connections from current node
        for node in range(len(capacities[current_node])):
            if node in visited or node in queue:
                continue

            # check if there is available capacity to node
            capacity = C[current_node][node]
            if capacity == 0:
                continue

            shortest_paths[node] = shortest_paths[current_node] + [node]
            if node == end:
                return shortest_paths[end]

            queue.append(node)
    return None


# Testing the function (from "a" to "f")
print(path_to_str(BFS(capacities, to_index("a"), to_index("f"))))


adf


In [5]:
def update_capacities(C: list, path: list) -> int:
    """Update all capacities along the path with appropriate flow value"""
    # find lowest capacity (bottleneck) along the path
    min_capacity = 999
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        capacity = C[u][v]
        if capacity < min_capacity:
            min_capacity = capacity
    # update with this capacity along the path
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        C[u][v] -= min_capacity
        C[v][u] += min_capacity  # backtracing flow

    return min_capacity


In [6]:
def edmonds_karp(C: list, source: int, sink: int):
    C = duplicate(C)  # duplicate to not alter original array
    max_flow = 0

    while True:
        # Find shortest path from source to sink
        path = BFS(C, source, sink)
        if path == None:
            # No more paths from source to sink, we are done
            break
        # Update all necessary capacities along the path
        flow = update_capacities(C, path)
        max_flow += flow  # increase max_flow accordingly
        print(f"{flow} : {path_to_str(path)}")

    # we are done, return the max flow
    return max_flow


max_flow = edmonds_karp(capacities, to_index("b"), to_index("f"))  # from b to f
print(f"Maximum Flow (b-f): {max_flow}")


4 : bdf
5 : bef
4 : badef
Maximum Flow (b-f): 13


In [7]:
# Finding maximum flow from everyone to everyone (used to verify your solution)
for i in range(len(capacities)):
    for j in range(len(capacities[i])):
        if i == j:
            continue
        max_flow = edmonds_karp(capacities, i, j)
        print(f"Maximum Flow ({to_str(i)}-{to_str(j)}) is {max_flow}")


9 : acb
Maximum Flow (a-b) is 9
9 : ac
3 : adec
4 : adfc
1 : adefc
Maximum Flow (a-c) is 17
9 : ad
4 : acbd
Maximum Flow (a-d) is 13
9 : ade
5 : acbe
Maximum Flow (a-e) is 14
4 : adf
5 : adef
4 : acbef
Maximum Flow (a-f) is 13
9 : ba
Maximum Flow (b-a) is 9
9 : bac
3 : bec
4 : bdfc
1 : befc
Maximum Flow (b-c) is 17
4 : bd
9 : bad
Maximum Flow (b-d) is 13
5 : be
4 : bde
5 : bade
Maximum Flow (b-e) is 14
4 : bdf
5 : bef
4 : badef
Maximum Flow (b-f) is 13
9 : cba
Maximum Flow (c-a) is 9
9 : cb
Maximum Flow (c-b) is 9
4 : cbd
5 : cbad
Maximum Flow (c-d) is 9
5 : cbe
4 : cbde
Maximum Flow (c-e) is 9
4 : cbdf
5 : cbef
Maximum Flow (c-f) is 9
3 : decba
4 : dfcba
1 : defcba
Maximum Flow (d-a) is 8
3 : decb
4 : dfcb
1 : defcb
Maximum Flow (d-b) is 8
3 : dec
4 : dfc
1 : defc
Maximum Flow (d-c) is 8
9 : de
4 : dfcbe
Maximum Flow (d-e) is 13
4 : df
9 : def
Maximum Flow (d-f) is 13
3 : ecba
5 : efcba
Maximum Flow (e-a) is 8
3 : ecb
5 : efcb
Maximum Flow (e-b) is 8
3 : ec
5 : efc
Maximum Flow (e-c) 